In [1]:
import pandas as pd
import numpy as np

import matplotlib.cm as cm
import matplotlib.colors as colors

# 1. Web scraping

Importing useful libraries for our web scraping process

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
# Official link provided in Assignment page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

# Due to changes in webpage, we will be using old revision of the above link
url = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=1011037969"

Web scraping process using [BeautifulSoap](https://www.crummy.com/software/BeautifulSoup/bs4/doc/) and [requests](https://requests.readthedocs.io/)

In [4]:
df = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])  # Columns for our csv file
r = requests.get(url)
soup = BeautifulSoup(r.text, "html.parser")
table = soup.find('table', {'class': 'wikitable sortable'}) # Find table with class name as 'wikitable, sortable'
for row in table.find_all('tr')[1:]:  # iterate over each table row
  code, borough, neighbor = [col.text.strip() for col in row.find_all('td')]  # extract table values
  if borough != "Not assigned":
    if neighbor == "Not assigned":
      neighbor = borough
    df = df.append({
        'PostalCode': code,
        'Borough': borough,
        'Neighborhood': neighbor
    }, ignore_index=True)

df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
len(df.PostalCode.unique())

103

In [6]:
df.shape

(103, 3)

Thus, Postal Code is unique for each records.

# 2. Geographical coordinates

Downloading the provided csv file that has Geographical coordinates of each postal code.

In [7]:
geospatial_df = pd.read_csv("https://cocl.us/Geospatial_data")
geospatial_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We will also be utilizing [Geocoder](https://geocoder.readthedocs.io/index.html) package to fetch geographical coordinates

In [8]:
!pip install geocoder -q
import geocoder

     |████████████████████████████████| 102kB 5.1MB/s 


There are tons of [geocode provider](https://geocoder.readthedocs.io/index.html#providers) supported by Geocoder package. But most of them has limitations, needs APIs, registration and others. Out of which, free providers are listed below.
- [ArcGIS](https://geocoder.readthedocs.io/providers/ArcGIS.html)
- [GeocodeFarm](https://geocoder.readthedocs.io/providers/GeocodeFarm.html)
- Komoot
- Location
- OSM

We will analyze and compare all of them to the provided Geographical coordinate that was previously downloaded.

Define a new Dataframe for compare various geocode provider

In [9]:
geocoder_df = pd.DataFrame()
i = 1
for code in df.PostalCode:
  print('\rProcessing:', i, end="")
  query = '{}, Toronto, Ontario'.format(code)
  row = [code]
  coders = [geocoder.arcgis, geocoder.geocodefarm, geocoder.komoot, geocoder.location, geocoder.osm]
  for c in coders:
    try:
      values = c(query).latlng
    except:
      values = [np.nan, np.nan]
    if values is None or len(values) !=2:
      values = [np.nan, np.nan]
    row.extend(values)
  row += list(geospatial_df[geospatial_df['Postal Code'] == code].to_numpy()[0])[1:]
  geocoder_df = geocoder_df.append([row], ignore_index=True)
  i += 1

Processing: 103

Define a valid columns as,

In [10]:
geocoder_df.columns = pd.MultiIndex.from_tuples((('PostalCode', ''),
                                             ("arcgis", "lat"), ("arcgis", "lng"),
                                             ("geocodefarm", "lat"), ("geocodefarm", "lng"),
                                             ("komoot", "lat"), ("komoot", "lng"),
                                             ("location", "lat"), ("location", "lng"),
                                             ("osm", "lat"), ("osm", "lng"),
                                             ("google", "lat"), ("google", "lng")))

geocoder_df.head()

PostalCode    arcgis            ...        osm     google           
                   lat       lng  ...        lng        lat        lng
0        M3A  43.75245 -79.32991  ... -79.383568  43.753259 -79.329656
1        M4A  43.73057 -79.31306  ...        NaN  43.725882 -79.315572
2        M5A  43.65512 -79.36264  ...        NaN  43.654260 -79.360636
3        M6A  43.72327 -79.45042  ...        NaN  43.718518 -79.464763
4        M7A  43.66253 -79.39188  ... -79.383568  43.662301 -79.389494

[5 rows x 13 columns]

In [11]:
geocoder_df.isna().sum()

PostalCode           0
arcgis       lat     0
             lng     0
geocodefarm  lat     0
             lng     0
komoot       lat     0
             lng     0
location     lat    83
             lng    83
osm          lat    83
             lng    83
google       lat     0
             lng     0
dtype: int64

There are 83 null entries for location and osm provider each. Hence, these providers are unreliable in terms of providing geological coordinates. Be mindful that GeocodeFarm provides limited request.

We can use Root Mean Squared Error to analyze the quality of our geocode provider as,

In [12]:
from sklearn.metrics import mean_squared_error

In [13]:
mean_squared_error(geocoder_df.google, geocoder_df.arcgis, squared=False)

0.017241480466064354

In [14]:
mean_squared_error(geocoder_df.google, geocoder_df.komoot, squared=False)

0.030215889135091226

In [15]:
mean_squared_error(geocoder_df.google, geocoder_df.geocodefarm, squared=False)

0.01739045143128333

Thus, ArcGIS and GeocodeFarm are the most reliable Geological coordinate provider among the free geocode providers.

Lets take average of those geological coordinate from ArcGIS and GeocodeFarm, and see some improvement in the result.

In [16]:
geocoder_df_concat = pd.concat((geocoder_df.geocodefarm, geocoder_df.arcgis))
mean_squared_error(geocoder_df.google, geocoder_df_concat.groupby(geocoder_df_concat.index).mean(), squared=False)

0.017235856388921184

Yes, there is a slight increase in quality of the Geological coordinate with the concatenated geocoder. We can stick with ArcGIS geocode provider for now. Lets use Geological coordinate provided by ArcGIS to create new columns in geocoder_df as mentioned in the task.

In [17]:
df[['Latitude', 'Longitude']] = geocoder_df['arcgis']
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


Successfully created new columns - Latitude, Longitude - in our Dataframe

# 3. Clustering Toronto

## Filtering out Toronto

In [18]:
toronto_df = df[df.Borough.str.contains('Toronto')].reset_index(drop=True)
toronto_df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
4,M4E,East Toronto,The Beaches,43.67709,-79.29547


In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(toronto_df['Borough'].unique()),
        len(toronto_df.Neighborhood.unique())))

The dataframe has 5 boroughs and 40 neighborhoods.


In [20]:
from geopy.geocoders import Nominatim
import folium

Geographical coordinates of Toronto

In [21]:
address = 'Toronto, Ontario'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Vizualizing Toronto neighborhoods.

In [22]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, neighborhood in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Borough'], toronto_df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Fetching Neighborood data using Foursquare API

Define Foursquare Credentials and Version

In [23]:
# @hidden_cell
%run foursquare_api.py
LIMIT = 100 # A default Foursquare API limit value

Lets use sample Neighborhood from Toronto

In [24]:
toronto_df.loc[0, 'Neighborhood']

'Regent Park, Harbourfront'

In [25]:
neighborhood_latitude = toronto_df.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.65512000000007, -79.36263999999994.


Let's get the top 100 venues that are in Regent Park, Harbourfront within a radius of 500 meters.

In [26]:
# @hidden_cell
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

In [27]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '604f71de1d22ba42e79b41f6'},
 'response': {'groups': [{'items': [{'reasons': {'count': 0,
       'items': [{'reasonName': 'globalInteractionReason',
         'summary': 'This spot is popular',
         'type': 'general'}]},
      'referralId': 'e-0-54ea41ad498e9a11e9e13308-0',
      'venue': {'categories': [{'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/bakery_',
          'suffix': '.png'},
         'id': '4bf58dd8d48988d16a941735',
         'name': 'Bakery',
         'pluralName': 'Bakeries',
         'primary': True,
         'shortName': 'Bakery'}],
       'id': '54ea41ad498e9a11e9e13308',
       'location': {'address': '362 King St E',
        'cc': 'CA',
        'city': 'Toronto',
        'country': 'Canada',
        'crossStreet': 'Trinity St',
        'distance': 192,
        'formattedAddress': ['362 King St E (Trinity St)',
         'Toronto ON M5A 1K9',
         'Canada'],
        'labeledLatLngs': [{'label': 'display',
 

In [28]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean the json and structure it into a pandas dataframe.

In [29]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues[filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Figs Breakfast & Lunch,Breakfast Spot,43.655675,-79.364503
3,Souvlaki Express,Greek Restaurant,43.655584,-79.364438
4,The Yoga Lounge,Yoga Studio,43.655515,-79.364955


In [30]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

23 venues were returned by Foursquare.


## Explore Neighborhoods in Toronto

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Utilizing above function on each neighborhood and create a new dataframe.

In [32]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighborhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Runnymede, The Junction, Weston-Pellam Park, Carlton Village
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West,  Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Har

In [33]:
print(toronto_venues.shape)
toronto_venues.head()

(1726, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,Souvlaki Express,43.655584,-79.364438,Greek Restaurant
4,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio


Number of venues for each neighborhood

In [34]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,65,65,65,65,65,65
"Brockton, Parkdale Village, Exhibition Place",84,84,84,84,84,84
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",100,100,100,100,100,100
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",79,79,79,79,79,79
Central Bay Street,61,61,61,61,61,61
Christie,10,10,10,10,10,10
Church and Wellesley,81,81,81,81,81,81
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,27,27,27,27,27,27


In [35]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 229 uniques categories.


## Analyze Each Neighborhood

In [36]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = ['Neighborhood'] + list(toronto_onehot.columns.drop('Neighborhood'))
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,...,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [37]:
toronto_onehot.shape

(1726, 229)

Group rows by neighborhood and take the mean of the frequency of occurrence of each category.

In [38]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Accessories Store,Adult Boutique,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Rental / Bike Share,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,...,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.015385,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.0,0.015385,0.046154,0.000000,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,0.00,0.015385,0.00,0.000000,0.000000,0.000000,0.000000,0.030769,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.0,0.000000,0.030769,...,0.000000,0.015385,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.015385,0.000000,0.000000,0.000000,0.000000,0.000000,0.015385
1,"Brockton, Parkdale Village, Exhibition Place",0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810,0.000000,0.011905,0.0,0.000000,0.023810,0.000000,0.059524,0.000000,0.011905,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.023810,0.011905,0.000000,0.000000,0.023810,0.000000,0.000000,0.000000,0.059524,0.0,0.000000,0.000000,...,0.000000,0.000000,0.00000,0.000000,0.011905,0.000000,0.000000,0.011905,0.00,0.011905,0.000000,0.000000,0.000000,0.011905,0.000000,0.000000,0.023810,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.011905,0.000000,0.00,0.000000,0.011905,0.000000,0.000000,0.000000,0.000000,0.000000,0.023810
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,0.020000,0.000000,0.0,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.010000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.040000,0.0,0.000000,0.000000,...,0.010000,0.000000,0.00000,0.010000,0.000000,0.000000,0.000000,0.000000,0.01,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.000000,0.000000,0.040000,0.000000,0.02,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.020000,0.020000,0.000000,0.000000,0.000000,0.00,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.0,0.000000,0.025316,0.012658,0.037975,0.000000,0.012658,0.000000,0.000000,0.012658,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.012658,0.012658,0.000000,0.000000,0.000000,0.000000,0.050633,0.0,0.012658,0.000000,...,0.000000,0.000000,0.00000,0.012658,0.000000,0.000000,0.000000,0.000000,0.00,0.012658,0.025316,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.012658,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012658
4,Central Bay Street,0.000000,0.000000,0.000000,0.000

In [39]:
toronto_grouped.shape

(40, 229)

Print each neighborhood along with the top 5 most common venues

In [40]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.05
2  Seafood Restaurant  0.05
3              Bakery  0.05
4          Restaurant  0.03


----Brockton, Parkdale Village, Exhibition Place----
         venue  freq
0         Café  0.06
1          Bar  0.06
2  Coffee Shop  0.05
3   Restaurant  0.05
4    Gift Shop  0.04


----Business reply mail Processing Centre, South Central Letter Processing Plant Toronto----
              venue  freq
0       Coffee Shop  0.07
1             Hotel  0.05
2  Sushi Restaurant  0.04
3              Café  0.04
4        Restaurant  0.03


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.06
2                Café  0.05
3   French Restaurant  0.04
4       Grocery Store  0.04


----Central Bay Street----
                       venue  freq
0                Coffee Shop  0

In [41]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Create a new dataframe and display the top 10 venues for each neighborhood.

In [42]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Breakfast Spot,Beer Bar,Pharmacy,Cheese Shop,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Café,Coffee Shop,Restaurant,Gift Shop,Sandwich Place,Nightclub,Lounge,Japanese Restaurant,Italian Restaurant
2,"Business reply mail Processing Centre, South C...",Coffee Shop,Hotel,Sushi Restaurant,Café,Restaurant,Taco Place,Japanese Restaurant,Sandwich Place,Salon / Barbershop,Salad Place
3,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Grocery Store,Bar,Park,French Restaurant,Gym / Fitness Center,Restaurant,Sandwich Place
4,Central Bay Street,Coffee Shop,Clothing Store,Bubble Tea Shop,Sandwich Place,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Plaza,Japanese Restaurant,Italian Restaurant


## Cluster Neighborhoods

Utilizing K-Means clustering algorithm to cluster the Neighborhood into 5 clusters

In [43]:
from sklearn.cluster import KMeans

In [44]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [45]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188,0,Coffee Shop,Sandwich Place,Burrito Place,Mediterranean Restaurant,Park,Fried Chicken Joint,Café,Italian Restaurant,Bank,Falafel Restaurant
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,0,Coffee Shop,Clothing Store,Italian Restaurant,Cosmetics Shop,Café,Hotel,Middle Eastern Restaurant,Japanese Restaurant,Movie Theater,Tanning Salon
3,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,0,Coffee Shop,Cosmetics Shop,Gastropub,Cocktail Bar,Clothing Store,Café,Italian Restaurant,Hotel,Park,Gym
4,M4E,East Toronto,The Beaches,43.67709,-79.29547,0,Health Food Store,Pub,Trail,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


Finally, let's visualize the resulting clusters.

In [46]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

### Cluster 1

In [47]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[2] + list(range(6,16))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Greek Restaurant,Italian Restaurant,Food Truck,Event Space,Electronics Store
1,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Burrito Place,Mediterranean Restaurant,Park,Fried Chicken Joint,Café,Italian Restaurant,Bank,Falafel Restaurant
2,"Garden District, Ryerson",Coffee Shop,Clothing Store,Italian Restaurant,Cosmetics Shop,Café,Hotel,Middle Eastern Restaurant,Japanese Restaurant,Movie Theater,Tanning Salon
3,St. James Town,Coffee Shop,Cosmetics Shop,Gastropub,Cocktail Bar,Clothing Store,Café,Italian Restaurant,Hotel,Park,Gym
4,The Beaches,Health Food Store,Pub,Trail,Dumpling Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
5,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Seafood Restaurant,Farmers Market,Breakfast Spot,Beer Bar,Pharmacy,Cheese Shop,Restaurant
6,Central Bay Street,Coffee Shop,Clothing Store,Bubble Tea Shop,Sandwich Place,Cosmetics Shop,Middle Eastern Restaurant,Hotel,Plaza,Japanese Restaurant,Italian Restaurant
7,Christie,Café,Grocery Store,Baby Store,Coffee Shop,Playground,Candy Store,Italian Restaurant,Yoga Studio,Escape Room,Ethiopian Restaurant
8,"Richmond, Adelaide, King",Coffee Shop,Hotel,Café,Restaurant,Gym,American Restaurant,Salad Place,Steakhouse,Asian Restaurant,Sushi Restaurant
9,"Dufferin, Dovercourt Village",Park,Smoke Shop,Bakery,Middle Eastern Restaurant,Brazilian Restaurant,Pool,Café,Bar,Bank,Pizza Place


### Cluster 2

In [48]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[2] + list(range(6,16))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Roselawn,Home Service,Yoga Studio,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


### Cluster 3

In [49]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[2] + list(range(6,16))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,"The Danforth West, Riverdale",Ice Cream Shop,Intersection,Bus Line,Business Service,Grocery Store,Park,Discount Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
18,Lawrence Park,Bus Line,Business Service,Swim School,Yoga Studio,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### Cluster 4

In [50]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[2] + list(range(6,16))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,"Runnymede, The Junction, Weston-Pellam Park, C...",Furniture / Home Store,Brewery,Business Service,Seafood Restaurant,Electronics Store,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


### Cluster 5

In [51]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[2] + list(range(6,16))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,"Forest Hill North & West, Forest Hill Road Park",Park,Lawyer,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
24,"North Toronto West, Lawrence Park",Playground,Gym Pool,Park,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
34,Rosedale,Park,Playground,Bike Trail,Yoga Studio,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
